In [2]:
import numpy as np 
import pandas as pd 
import os

In [188]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
holiday_data = pd.read_csv("holidays_events.csv")
oil_data = pd.read_csv("oil.csv")
stores_data = pd.read_csv("stores.csv")
transactions_data = pd.read_csv("transactions.csv")

In [3]:
train_data

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [4]:
print(train_data.isnull().sum())

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64


In [7]:
print(test_data.isnull().sum())

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64


In [5]:
holiday_data

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [112]:
holiday_data.columns

Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')

In [ ]:
holiday_data["type"].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [110]:
holiday_data["locale"].unique()

array(['Local', 'Regional', 'National'], dtype=object)

In [111]:
holiday_data["locale_name"].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato',
       'Ibarra', 'Quevedo', 'Santo Domingo de los Tsachilas',
       'Santa Elena', 'Quito', 'Loja', 'Salinas', 'Guayaquil'],
      dtype=object)

In [7]:
print(holiday_data.isnull().sum())

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64


In [8]:
oil_data

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [11]:
print(oil_data.isnull().sum())

date           0
dcoilwtico    43
dtype: int64


In [14]:
print(stores_data.isnull().sum())

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64


In [15]:
transactions_data

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [16]:
print(transactions_data.isnull().sum())

date            0
store_nbr       0
transactions    0
dtype: int64


### Handling missing values

In [189]:
oil_data['dcoilwtico'].ffill(inplace=True) 
oil_data['dcoilwtico'].bfill(inplace=True) 

In [122]:
print(oil_data.isnull().sum())

date          0
dcoilwtico    0
dtype: int64


### Holiday data

In [190]:
# National Holidays (merged on date) 
national_holidays = holiday_data[holiday_data['locale'] == 'National'].copy()
national_holidays['is_national_holiday'] = 1
# Handle transferred days correctly
national_holidays.loc[national_holidays['transferred'] == True, 'is_national_holiday'] = 0
national_holidays = national_holidays[['date', 'is_national_holiday']]
national_holidays.drop_duplicates(inplace=True)

In [191]:
# Regional Holidays (merged on date and state) ---
regional_holidays = holiday_data[holiday_data['locale'] == 'Regional'].copy()
# Rename 'locale_name' to 'state' to allow for a correct merge
regional_holidays.rename(columns={'locale_name': 'state'}, inplace=True)
regional_holidays['is_regional_holiday'] = 1
regional_holidays.loc[regional_holidays['transferred'] == True, 'is_regional_holiday'] = 0
regional_holidays = regional_holidays[['date', 'state', 'is_regional_holiday']]
regional_holidays.drop_duplicates(inplace=True)

In [193]:
# Local Holidays (merged on date and city) ---
local_holidays = holiday_data[holiday_data['locale'] == 'Local'].copy()
# Rename 'locale_name' to 'city' to allow for a correct merge
local_holidays.rename(columns={'locale_name': 'city'}, inplace=True)
local_holidays['is_local_holiday'] = 1
local_holidays.loc[local_holidays['transferred'] == True, 'is_local_holiday'] = 0
local_holidays = local_holidays[['date', 'city', 'is_local_holiday']]
local_holidays.drop_duplicates(inplace=True)

### Merging all other dataframes into train and test data frames

In [194]:
train_data = pd.merge(train_data, stores_data, on='store_nbr', how='left')
train_data = pd.merge(train_data, transactions_data, on=['date', 'store_nbr'], how='left')
train_data = pd.merge(train_data, oil_data, on='date', how='left')
train_data['transactions'].fillna(0, inplace=True)
train_data['dcoilwtico'] = train_data['dcoilwtico'].ffill()

In [195]:
train_data = pd.merge(train_data, national_holidays, on='date', how='left')
train_data = pd.merge(train_data, regional_holidays, on=['date', 'state'], how='left')
train_data = pd.merge(train_data, local_holidays, on=['date', 'city'], how='left')

train_data['is_national_holiday'].fillna(0, inplace=True)
train_data['is_regional_holiday'].fillna(0, inplace=True)
train_data['is_local_holiday'].fillna(0, inplace=True)

In [160]:
train_data.head(10)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions,dcoilwtico,is_national_holiday,is_regional_holiday,is_local_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
6,6,2013-01-01,1,CELEBRATION,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
7,7,2013-01-01,1,CLEANING,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
8,8,2013-01-01,1,DAIRY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0
9,9,2013-01-01,1,DELI,0.0,0,Quito,Pichincha,D,13,NaN,93.14,1.0,0.0,0.0


In [196]:
test_data = pd.merge(test_data, stores_data, on='store_nbr', how='left')
test_data = pd.merge(test_data, transactions_data, on=['date', 'store_nbr'], how='left')
test_data = pd.merge(test_data, oil_data, on='date', how='left')
test_data['transactions'].fillna(0, inplace=True)
test_data['dcoilwtico'] = test_data['dcoilwtico'].ffill()

In [197]:
test_data = pd.merge(test_data, national_holidays, on='date', how='left')
test_data = pd.merge(test_data, regional_holidays, on=['date', 'state'], how='left')
test_data = pd.merge(test_data, local_holidays, on=['date', 'city'], how='left')

test_data['is_national_holiday'].fillna(0, inplace=True)
test_data['is_regional_holiday'].fillna(0, inplace=True)
test_data['is_local_holiday'].fillna(0, inplace=True)

In [163]:
train_data.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'city',
       'state', 'type', 'cluster', 'transactions', 'dcoilwtico',
       'is_national_holiday', 'is_regional_holiday', 'is_local_holiday'],
      dtype='object')

In [198]:
print(train_data.shape)
print(test_data.shape)

(3000888, 15)
(28512, 14)


## Feature Engineering

### Basic date based features 

In [199]:
train_data["date"] = pd.to_datetime(train_data["date"])
test_data["date"] = pd.to_datetime(test_data["date"])

In [200]:
train_data['year'] = train_data['date'].dt.year
train_data['dayofyear'] = train_data['date'].dt.dayofyear
train_data['quarter'] = train_data['date'].dt.quarter
train_data['is_weekend'] = (train_data['date'].dt.dayofweek >= 5).astype(int)

test_data['year'] = test_data['date'].dt.year
test_data['dayofyear'] = test_data['date'].dt.dayofyear
test_data['quarter'] = test_data['date'].dt.quarter
test_data['is_weekend'] = (test_data['date'].dt.dayofweek >= 5).astype(int)


### Cyclic features with Sin & Cos transformation

In [201]:
train_data['month_sin'] = np.sin(2 * np.pi * train_data['date'].dt.month / 12)
train_data['month_cos'] = np.cos(2 * np.pi * train_data['date'].dt.month / 12)
train_data['dayofweek_sin'] = np.sin(2 * np.pi * train_data['date'].dt.dayofweek / 7)
train_data['dayofweek_cos'] = np.cos(2 * np.pi * train_data['date'].dt.dayofweek / 7)

test_data['month_sin'] = np.sin(2 * np.pi * test_data['date'].dt.month / 12)
test_data['month_cos'] = np.cos(2 * np.pi * test_data['date'].dt.month / 12)
test_data['dayofweek_sin'] = np.sin(2 * np.pi * test_data['date'].dt.dayofweek / 7)
test_data['dayofweek_cos'] = np.cos(2 * np.pi * test_data['date'].dt.dayofweek / 7)

#### Holiday & Special Event Features

In [202]:
train_data['is_holiday'] = train_data[['is_national_holiday', 'is_regional_holiday', 'is_local_holiday']].max(axis=1)
test_data['is_holiday'] = test_data[['is_national_holiday', 'is_regional_holiday', 'is_local_holiday']].max(axis=1)

### Payday Feature

In [203]:
# Wages are paid on the 15th and last day of the month
train_data['is_payday'] = np.where(
    (train_data['date'].dt.day == 15) | (train_data['date'].dt.is_month_end),
    1, 0
)

test_data['is_payday'] = np.where(
    (test_data['date'].dt.day == 15) | (test_data['date'].dt.is_month_end),
    1, 0
)

### Earthquake feature

In [204]:
# Creating a flag for few week after earthquake event in 2016
earthquake_start = pd.to_datetime('2016-04-16')
earthquake_end = pd.to_datetime('2016-05-31')
train_data['post_earthquake_effect'] = np.where(
    (train_data['date'] >= earthquake_start) & (train_data['date'] <= earthquake_end),
    1, 0
)
test_data['post_earthquake_effect'] = np.where(
    (test_data['date'] >= earthquake_start) & (test_data['date'] <= earthquake_end),
    1, 0
)

In [205]:
print(train_data.shape)
print(test_data.shape)

(3000888, 26)
(28512, 25)


In [206]:
print(train_data.isnull().sum())

id                        0
date                      0
store_nbr                 0
family                    0
sales                     0
onpromotion               0
city                      0
state                     0
type                      0
cluster                   0
transactions              0
dcoilwtico                0
is_national_holiday       0
is_regional_holiday       0
is_local_holiday          0
year                      0
dayofyear                 0
quarter                   0
is_weekend                0
month_sin                 0
month_cos                 0
dayofweek_sin             0
dayofweek_cos             0
is_holiday                0
is_payday                 0
post_earthquake_effect    0
dtype: int64


In [209]:
print(test_data.isnull().sum())

id                            0
date                          0
store_nbr                     0
family                        0
onpromotion                   0
city                          0
state                         0
type                          0
cluster                       0
transactions                  0
dcoilwtico                    0
is_national_holiday           0
is_regional_holiday           0
is_local_holiday              0
year                          0
dayofyear                     0
quarter                       0
is_weekend                    0
month_sin                     0
month_cos                     0
dayofweek_sin                 0
dayofweek_cos                 0
is_holiday                    0
is_payday                     0
post_earthquake_effect        0
source                        0
sales                     28512
dtype: int64


### Lag features

#### The test data is for a 16-day period (from Aug 16 to Aug 31).So we cannot create any lag features lower than 16 days since those data is not available 

#### Before creating any lag or rolling features we need to merge the train_data and test_data 

In [207]:
# Add an identifier column to distinguish between train and test sets
train_data['source'] = 'train'
test_data['source'] = 'test'
test_data['sales'] = np.nan
df = pd.concat([train_data, test_data], ignore_index=True, sort=False)

In [208]:
lags = [16, 21, 28]
for lag in lags:
    df[f'sales_lag_{lag}'] = df.groupby(['store_nbr', 'family'])['sales'].shift(lag)

windows = [7, 14, 28]
for window in windows:
    rolling_series_mean = df.groupby(['store_nbr', 'family'])[f'sales_lag_16'].rolling(window).mean()
    rolling_series_std = df.groupby(['store_nbr', 'family'])[f'sales_lag_16'].rolling(window).std()

    df[f'sales_rolling_mean_{window}'] = rolling_series_mean.reset_index(level=[0,1], drop=True)
    df[f'sales_rolling_std_{window}'] = rolling_series_std.reset_index(level=[0,1], drop=True)




In [210]:
df.shape

(3029400, 36)

In [211]:
print(df.isnull().sum())

id                            0
date                          0
store_nbr                     0
family                        0
sales                     28512
onpromotion                   0
city                          0
state                         0
type                          0
cluster                       0
transactions                  0
dcoilwtico                    0
is_national_holiday           0
is_regional_holiday           0
is_local_holiday              0
year                          0
dayofyear                     0
quarter                       0
is_weekend                    0
month_sin                     0
month_cos                     0
dayofweek_sin                 0
dayofweek_cos                 0
is_holiday                    0
is_payday                     0
post_earthquake_effect        0
source                        0
sales_lag_16              28512
sales_lag_21              37422
sales_lag_28              49896
sales_rolling_mean_7      39204
sales_ro

In [212]:
df.fillna(0, inplace=True)

In [213]:
print(df.isnull().sum())

id                        0
date                      0
store_nbr                 0
family                    0
sales                     0
onpromotion               0
city                      0
state                     0
type                      0
cluster                   0
transactions              0
dcoilwtico                0
is_national_holiday       0
is_regional_holiday       0
is_local_holiday          0
year                      0
dayofyear                 0
quarter                   0
is_weekend                0
month_sin                 0
month_cos                 0
dayofweek_sin             0
dayofweek_cos             0
is_holiday                0
is_payday                 0
post_earthquake_effect    0
source                    0
sales_lag_16              0
sales_lag_21              0
sales_lag_28              0
sales_rolling_mean_7      0
sales_rolling_std_7       0
sales_rolling_mean_14     0
sales_rolling_std_14      0
sales_rolling_mean_28     0
sales_rolling_std_28

### Handling categorical features

In [104]:
df.dtypes

id                                 int64
date                      datetime64[ns]
store_nbr                          int64
family                            object
sales                            float64
onpromotion                        int64
city                              object
state                             object
store_type                        object
cluster                            int64
transactions                     float64
dcoilwtico                       float64
year                               int32
dayofyear                          int32
quarter                            int32
is_weekend                         int32
month_sin                        float64
month_cos                        float64
dayofweek_sin                    float64
dayofweek_cos                    float64
is_holiday_day                     int32
is_work_day                        int32
is_payday                          int32
post_earthquake_effect             int32
source          

In [106]:
df["family"].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [107]:
df["city"].unique()

array(['Quito', 'Cayambe', 'Latacunga', 'Riobamba', 'Ibarra',
       'Santo Domingo', 'Guaranda', 'Puyo', 'Ambato', 'Guayaquil',
       'Salinas', 'Daule', 'Babahoyo', 'Quevedo', 'Playas', 'Libertad',
       'Cuenca', 'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen'],
      dtype=object)

In [105]:
df["state"].unique()

array(['Pichincha', 'Cotopaxi', 'Chimborazo', 'Imbabura',
       'Santo Domingo de los Tsachilas', 'Bolivar', 'Pastaza',
       'Tungurahua', 'Guayas', 'Santa Elena', 'Los Rios', 'Azuay', 'Loja',
       'El Oro', 'Esmeraldas', 'Manabi'], dtype=object)

In [108]:
df["store_type"].unique()

array(['D', 'C', 'B', 'E', 'A'], dtype=object)

In [152]:
df.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'city',
       'state', 'type', 'cluster', 'transactions', 'dcoilwtico',
       'is_national_holiday', 'is_regional_holiday', 'is_local_holiday',
       'year', 'dayofyear', 'quarter', 'is_weekend', 'month_sin', 'month_cos',
       'dayofweek_sin', 'dayofweek_cos', 'is_holiday', 'is_payday',
       'post_earthquake_effect', 'source', 'sales_lag_16', 'sales_lag_21',
       'sales_lag_28', 'sales_rolling_mean_7', 'sales_rolling_std_7',
       'sales_rolling_mean_14', 'sales_rolling_std_14',
       'sales_rolling_mean_28', 'sales_rolling_std_28'],
      dtype='object')

#### Converting columns to 'category' dtype

In [214]:
categorical_cols = [
    'store_nbr', 
    'family', 
    'city', 
    'state', 
    'type', 
    'cluster'
]
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [181]:
df.dtypes

id                                 int64
date                      datetime64[ns]
store_nbr                       category
family                          category
sales                            float64
onpromotion                        int64
city                            category
state                           category
type                            category
cluster                         category
transactions                     float64
dcoilwtico                       float64
is_national_holiday              float64
is_regional_holiday              float64
is_local_holiday                 float64
year                               int32
dayofyear                          int32
quarter                            int32
is_weekend                         int32
month_sin                        float64
month_cos                        float64
dayofweek_sin                    float64
dayofweek_cos                    float64
is_holiday                       float64
is_payday       

### Separation of data again to train and test

In [215]:
train_final = df[df['source'] == 'train'].copy()
test_final = df[df['source'] == 'test'].copy()

# Drop the helper columns
train_final.drop(columns=['source'], inplace=True)
test_final.drop(columns=['source'], inplace=True)
test_final.drop(columns=['sales'], inplace=True, errors='ignore')


In [218]:
print(train_final.isnull().sum())


id                        0
date                      0
store_nbr                 0
family                    0
sales                     0
onpromotion               0
city                      0
state                     0
type                      0
cluster                   0
transactions              0
dcoilwtico                0
is_national_holiday       0
is_regional_holiday       0
is_local_holiday          0
year                      0
dayofyear                 0
quarter                   0
is_weekend                0
month_sin                 0
month_cos                 0
dayofweek_sin             0
dayofweek_cos             0
is_holiday                0
is_payday                 0
post_earthquake_effect    0
sales_lag_16              0
sales_lag_21              0
sales_lag_28              0
sales_rolling_mean_7      0
sales_rolling_std_7       0
sales_rolling_mean_14     0
sales_rolling_std_14      0
sales_rolling_mean_28     0
sales_rolling_std_28      0
dtype: int64


In [219]:
test_final.isnull().sum()

id                        0
date                      0
store_nbr                 0
family                    0
onpromotion               0
city                      0
state                     0
type                      0
cluster                   0
transactions              0
dcoilwtico                0
is_national_holiday       0
is_regional_holiday       0
is_local_holiday          0
year                      0
dayofyear                 0
quarter                   0
is_weekend                0
month_sin                 0
month_cos                 0
dayofweek_sin             0
dayofweek_cos             0
is_holiday                0
is_payday                 0
post_earthquake_effect    0
sales_lag_16              0
sales_lag_21              0
sales_lag_28              0
sales_rolling_mean_7      0
sales_rolling_std_7       0
sales_rolling_mean_14     0
sales_rolling_std_14      0
sales_rolling_mean_28     0
sales_rolling_std_28      0
dtype: int64

In [220]:
print(train_final.shape)
print(test_final.shape)

(3000888, 35)
(28512, 34)
